# TP Représentation symbolique du langage naturel et inférence

In [1]:
# This patch is needed. It holds 2 fixes to skolemization and answers.
import mynltk
import nltk
from nltk.sem import Expression, skolemize
from nltk.inference.resolution import clausify, Clause

import os
os.environ['PROVER9'] = 'c:/Program Files (x86)/Prover9-Mace4/bin-win32/'

read_expr = nltk.sem.Expression.fromstring

Loading patch
Patching nltk.sem.logic


NLTK permet de lire des expressions logiques et de les transformer. 

Examinons le résultat de la skolemisation :

In [2]:
from nltk.sem.logic import skolem_function
print(skolem_function())
print(skolemize(Expression.fromstring('exists x. happy(x)')))
print(skolemize(Expression.fromstring('all x. happy(x)')))
print(skolemize(Expression.fromstring('- exists x. happy(x)')))
print(skolemize(Expression.fromstring('- all x. happy(x)')))
print(skolemize(Expression.fromstring('all x. exists y. from(x, y)')))

F1
happy(F2)
happy(z3)
-happy(z4)
-happy(F5)
from(z7,F6(z7))


On note l'existence d'un compteur pour les fonctions de Skolem incrémenté à chaque utilisation.

## Démonstration par résolution

NLTK fournit un démonstrateur de théorèmes par résolution.
Nous pouvons illustrer son fonctionnement par l'exemple suivant :

> The law says that it is a crime for an American to sell weapons to hostile nations.
>  
> The country Nono, an enemy of America, has some missiles, 
> and all of its missiles were sold to it by Colonel West, who is American.

(Exemple fourni par Russell & Norvig, "Artificial Intelligence", chapitre 9)

La traduction en formules logiques de ces énoncés donne :

$\forall x y z \; american(x) \wedge weapon(y) \wedge sells(x, y, z) \wedge hostile(z) \rightarrow criminal(x)$

$\forall x \; missile(x) \wedge owns(Nono, x) \rightarrow sells(West, x, Nono)$

$\forall x \; ennemy(x, America) \rightarrow hostile(x)$

$\forall x \; missile(x) \rightarrow weapon(x)$

$american(West) \qquad ennemy(Nono, America)$

$\exists x \; missile(x) \wedge owns(Nono, x)$


In [3]:
c1 = read_expr('all x y z.(american(x) & weapon(y) & sells(x, y, z) & hostile(z) -> criminal(x))')
c2 = read_expr('all x.(missile(x) & owns(Nono, x) -> sells(West, x, Nono))')
c3 = read_expr('all x. (ennemy(x, America) -> hostile(x))')
c4 = read_expr('all x.(missile(x) -> weapon(x))')
c5 = read_expr('american(West)')
c6 = read_expr('ennemy(Nono, America)')
c7 = read_expr('some x.(missile(x) & owns(Nono, x))')

NLTK sait transformer ces formules sous forme clausale :

In [4]:
clausify(c1)

[{-american(z13), -weapon(z11), -sells(z13,z11,z12), -hostile(z12), criminal(z13)}]

In [5]:
clausify(c7)
# nltk.sem.skolemize(c7)

[{missile(F14)}, {owns(Nono,F14)}]

In [6]:
conclusion = read_expr('criminal(West)')
# L'utilisation du prover intégré à NLTK est extrêmement lente.
# prover = nltk.ResolutionProverCommand(conclusion, [c1,c2,c3,c4,c5,c6,c7])
# On préfère utiliser Prover9.
prover = nltk.Prover9Command(conclusion, [c1,c2,c3,c4,c5,c6,c7])
print(prover.prove())
print(prover.proof())

True
============================== prooftrans ============================
Prover9 (32) version Dec-2007, Dec 2007.
Process 13428 was started by Maamar on DESKTOP-VUU0N4G,
Wed Nov 15 21:52:24 2023
The command was "/cygdrive/c/Program Files (x86)/Prover9-Mace4/bin-win32/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 24.
% Level of proof is 6.
% Maximum clause weight is 0.
% Given clauses 0.

1 (all x all y all z (american(x) & weapon(y) & sells(x,y,z) & hostile(z) -> criminal(x))).  [assumption].
2 (all x (missile(x) & owns(Nono,x) -> sells(West,x,Nono))).  [assumption].
3 (all x (ennemy(x,America) -> hostile(x))).  [assumption].
4 (all x (missile(x) -> weapon(x))).  [assumption].
5 (exists x (missile(x) &

La clause vide a été dérivée. 

On peut demander à examiner les hypothèses :

In [7]:
for a in prover.assumptions():
    print(a)

all x y z.((american(x) & weapon(y) & sells(x,y,z) & hostile(z)) -> criminal(x))
all x.((missile(x) & owns(Nono,x)) -> sells(West,x,Nono))
all x.(ennemy(x,America) -> hostile(x))
all x.(missile(x) -> weapon(x))
american(West)
ennemy(Nono,America)
exists x.(missile(x) & owns(Nono,x))


Il est possible d'extraire des réponses d'un arbre de preuve. La clause vide n'étant pas un moyen de véhiculer l'information que l'on souhaite récupérer, le calcul d'une réponse passe par l'utilisation d'un littéral spécifique ANSWER qui est ignoré par l'algorithme de résolution, mais qui mémorise la variable contenant a réponse cherchée.

## Travail

Résoudre le problème suivant :

>Everyone who loves all animals is loved by someone.
>
>Anyone who kills an animal is loved by no one. 
>
>Jack loves all animals. 
>
>Either Jack or Curiosity killed the cat, who is named Tuna. 
>
>Did Curiosity kill the cat?


Commençons par traduire les énoncés en logique du premier ordre :

1. **Tout le monde qui aime tous les animaux est aimé par quelqu'un.**

   $\forall x \left( \left( \forall y \, \text{animal}(y) \rightarrow \text{aime}(x,y) \right) \rightarrow \exists z \, \text{aime}(z,x) \right)$

2. **Quiconque tue un animal n'est aimé de personne.**
   
   $\forall x \left( \exists y \left( \text{animal}(y) \wedge \text{tue}(x,y) \right) \rightarrow \neg \exists z \, \text{aime}(z,x) \right)$
   

3. **Jack aime tous les animaux.**
   
   $\forall y \, \text{animal}(y) \rightarrow \text{aime}(\text{Jack},y)$

4. **Soit Jack, soit Curiosity a tué le chat, nommé Tuna.**
   
   $\text{tue}(\text{Jack},\text{Tuna}) \vee \text{tue}(\text{Curiosity},\text{Tuna})$
   
   
   $\text{animal}(\text{Tuna}) \wedge \text{chat}(\text{Tuna})$


5. **Curiosity a-t-il tué le chat?**
   
   $\text{tue}(\text{Curiosity},\text{Tuna})$
   


In [8]:
c1 = read_expr('all x. (all y.(animal(y) -> love(x,y)) -> exists z. love(z,x))')
c2 = read_expr('all x. ((exists y. (animal(y) & kill(x,y))) -> all z. -love(z,x))')
c3 = read_expr('all x. (animal(x) -> love(Jack,x))')
c4 = read_expr('all x. (cat(x) -> animal(x))')
c5 = read_expr('cat(Tuna)')
c6 = read_expr('(kill(Jack, Tuna) & -kill(Curiosity, Tuna)) | (kill(Curiosity, Tuna) & -kill(Jack, Tuna))')
conclusion = read_expr('kill(Curiosity, Tuna)')

prover = nltk.inference.Prover9Command(conclusion, [c1,c2,c3,c4,c5,c6])
print(prover.prove())
print(prover.proof())

True
============================== prooftrans ============================
Prover9 (32) version Dec-2007, Dec 2007.
Process 7944 was started by Maamar on DESKTOP-VUU0N4G,
Wed Nov 15 22:12:22 2023
The command was "/cygdrive/c/Program Files (x86)/Prover9-Mace4/bin-win32/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.01) seconds.
% Length of proof is 21.
% Level of proof is 6.
% Maximum clause weight is 8.
% Given clauses 9.

1 (all x ((all y (animal(y) -> love(x,y))) -> (exists z love(z,x)))).  [assumption].
2 (all x ((exists y (animal(y) & kill(x,y))) -> (all z -love(z,x)))).  [assumption].
3 (all x (animal(x) -> love(Jack,x))).  [assumption].
4 (all x (cat(x) -> animal(x))).  [assumption].
5 kill(Jack,Tuna) & -kill(Curiosity,Tuna) | kill(

La preuve générée par Prover9 indique que la conclusion kill(Curiosity, Tuna) est vraie en se basant sur les formules logiques que nous avons fournies. Voici comment interpréter les étapes clés de la preuve :

Hypothèses et Objectif :

Les hypothèses (clauses 1 à 5 et 12) sont les formules logiques que nous avons définies.
L'objectif est de prouver que kill(Curiosity, Tuna) (clause 6).

Processus de Raisonnement :

Le système utilise diverses techniques de résolution (comme clausify, resolve, back_unit_del) pour manipuler et combiner les hypothèses.
Les clauses numérotées (7 à 27) représentent les étapes intermédiaires et les conclusions tirées de ces manipulations.

Résultat Final :

La clause $F (False) est obtenue à la fin (clause 27), ce qui dans le contexte de la logique de Prover9 signifie que la contradiction a été trouvée dans la négation de l'objectif. Cela indique que l'objectif (c'est-à-dire kill(Curiosity, Tuna)) est vrai en raison de la contradiction trouvée en supposant le contraire.

Interprétation :

Selon les formules logiques et les règles que nous avons fournies, il est logiquement valide de conclure que Curiosity a tué le chat, Tuna.

Considérations Importantes :

En résumé, la preuve fournie par Prover9 confirme que, selon votre modélisation logique du scénario, Curiosity est responsable de la mort du chat, Tuna.
Globalement, l'approche est correcte. Cependant, j'ai décidé de tester également le contraire de la conclusion (par exemple, kill(Jack, Tuna)) pour voir si le système peut également déduire cette conclusion ou non, afin d'avoir une compréhension complète du scénario.

In [9]:
# Nouvelle conclusion à tester
new_conclusion = read_expr('kill(Jack, Tuna)')

# Utiliser Prover9 pour tester la nouvelle conclusion
new_prover = nltk.inference.Prover9Command(new_conclusion, [c1, c2, c3, c4, c5, c6])
print(new_prover.prove())
print(new_prover.proof())


False
============================== prooftrans ============================
Prover9 (32) version Dec-2007, Dec 2007.
Process 6464 was started by Maamar on DESKTOP-VUU0N4G,
Wed Nov 15 22:21:16 2023
The command was "/cygdrive/c/Program Files (x86)/Prover9-Mace4/bin-win32/prover9".
============================== end of head ===========================

============================== end of input ==========================


Le résultat False indique que, selon les formules logiques que nous avons fournies, il n'est pas possible de déduire logiquement que Jack a tué Tuna. Ce résultat, en combinaison avec le résultat précédent (où il a été déduit que Curiosity a tué Tuna), suggère que notre modèle logique est suffisamment spécifique pour identifier un coupable unique dans ce scénario.

En résumé, notre modèle logique mène à la conclusion que Curiosity, et non Jack, est responsable de la mort du chat Tuna, selon les informations et les règles que nous avons établies. Cela démontre une bonne cohérence dans notre modèle et confirme que les prémisses conduisent à une conclusion unique et spécifique dans ce cas.